Before running this code, you need to set up the Earth Engine Python API.
Follow these steps (based on official Google documentation):
1. Install the Earth Engine API if not already installed (e.g., in a terminal: pip install earthengine-api --upgrade).
2. Create a Google Cloud Project: Go to https://console.cloud.google.com/projectcreate and create a new project. Note your Project ID (e.g., 'my-earth-engine-project').
3. Enable the Earth Engine API: In your Google Cloud Console, search for "Earth Engine API" and enable it for your project.
4. Authenticate: Run ee.Authenticate() below if prompted; this will open a browser for Google sign-in and generate a token.
5. Initialize: Replace 'your-project-id' below with your actual Google Cloud Project ID.
For more details, see: https://developers.google.com/earth-engine/guides/python_install

# Initialization

In [1]:
# Import necessary libraries (assuming they are installed and Earth Engine is authenticated)
# Note: This code requires the 'ee' (Google Earth Engine) and 'geemap' libraries.
# Earth Engine authentication is needed: ee.Authenticate() and ee.Initialize() should be run beforehand.

#import relevant libraries
import ee
import geemap
import geopandas as gpd
import json


In [2]:
#Authenticate and initilaize GEE 
try:
    ee.Initialize(project='uss-dissertation')
except Exception:
    print("GEE initialization failed. Run ee.Authenticate() and try again.")
    ee.Authenticate()
    ee.Initialize(project='uss-dissertation')

# Calculations

In [ ]:

# Load city boundary from a Google Earth Engine asset
def load_city_boundary(asset_id):
    try:
        city_boundary = ee.FeatureCollection(asset_id)
        print("City boundary loaded successfully!")
        return city_boundary
    except Exception as e:
        print(f"Failed to load city boundary: {str(e)}")
        return None

# Calculate LST for given years using Landsat data
def calculate_lst(city_boundary, years):

    lst_layers = {}
    for year in years:
       # Processes summer months (June-Aug) to capture peak heat periods.
        start_date = f'{year}-06-01'
        end_date = f'{year}-08-31'
        
        # Landsat 8/9 Level-2 for LST
        # Note: Using Level-2 data which includes atmospherically corrected surface temperature in the ST_B10 band.
        landsat = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                   .filterBounds(city_boundary.geometry())
                   .filterDate(start_date, end_date)
                   .filter(ee.Filter.lt('CLOUD_COVER', 5)))
        #Merges Landsat 8 and 9 for better coverage
        landsat = landsat.merge(
            ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
            .filterBounds(city_boundary.geometry())
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lt('CLOUD_COVER', 5)) #Filters for low cloud cover (<5%) to ensure data quality.
        )
        # This logs details for each image, including spacecraft, path/row, date, and ID, for verification and reproducibility.
        landsat_size = landsat.size().getInfo()
        print(f"Number of Landsat images for {year}: {landsat_size}")
        if landsat_size > 0:
            image_list = landsat.toList(landsat.size())
            for i in range(landsat_size):
                image = ee.Image(image_list.get(i))
                spacecraft = image.get('SPACECRAFT_ID').getInfo()
                path = image.get('WRS_PATH').getInfo()
                row = image.get('WRS_ROW').getInfo()
                date_acquired = image.get('DATE_ACQUIRED').getInfo()
                image_id = image.get('system:id').getInfo()
                print(f"Image {i+1}: {spacecraft} - Path: {path} - Row: {row} - Date: {date_acquired} - ID: {image_id}")
        
        # Compute mean composite of LST over the filtered images for the period.
        landsat = landsat.mean()
        
        # Derive LST from the ST_B10 band
        # Note: The following chain processes the surface temperature band:
        # - select('ST_B10'): Chooses the thermal band containing surface temperature data (in digital numbers).
        # - multiply(ee.Image.constant(0.00341802)): Applies the scale factor to convert to Kelvin (per USGS Collection 2 guidelines).
        # - add(ee.Image.constant(149.0)): Adds the offset to complete the Kelvin conversion.
        # - subtract(ee.Image.constant(273.15)): Converts Kelvin to Celsius for standard temperature units.
        # - clip(city_boundary.geometry()): Restricts the image to the city boundary to focus the analysis.
        # - unmask(0): Replaces any masked (no-data) pixels with 0 to ensure a complete raster for visualization.
        lst = landsat.select('ST_B10').multiply(ee.Image.constant(0.00341802)).add(ee.Image.constant(149.0)).subtract(ee.Image.constant(273.15)).clip(city_boundary.geometry()).unmask(0)
        lst_layers[year] = lst
        print(f"LST for {year} calculated successfully!")
    return lst_layers

# Add multi-year LST layers to the map
def add_multi_year_layers(map_obj, city_boundary, lst_layers):
    # Note: Palette chosen for temperature gradient: cool (blues) to hot (reds).
    lst_palette = ['darkblue', 'blue', 'cyan', 'green', 'yellow', 'orange', 'red', 'darkred']
    lst_images = [lst_layers[year] for year in lst_layers if lst_layers[year] is not None]
    if lst_images:
        lst_collection = ee.ImageCollection(lst_images)
        # Computes overall min/max across all years for consistent visualization scaling.
        lst_stats = lst_collection.reduce(ee.Reducer.minMax()).reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=city_boundary.geometry(),
            scale=30,
            maxPixels=1e9
        ).getInfo()
        lst_min = lst_stats.get('ST_B10_min_min', 20)
        lst_max = lst_stats.get('ST_B10_max_max', 60)
        print(f"Calculated LST Range: Min {lst_min:.2f}°C, Max {lst_max:.2f}°C")
        lst_vis = {"min": lst_min, "max": lst_max, "palette": lst_palette}
    else:
        lst_vis = {"min": 20, "max": 60, "palette": lst_palette}
        print("No valid LST data; using fallback range: 20–60°C")
    
    # Add layers for each year
    # Note: Each year's LST is added as a separate layer for toggling and comparison. (useful for inital exploration when i had multiple years)
    for year in years:
        if lst_layers.get(year) is not None:
            map_obj.addLayer(lst_layers[year], lst_vis, f'LST_{year}')
    
    return map_obj

# Load and map
# Asset ID specific to Riyadh boundary; adjust if needed for other cities.
asset_id = 'projects/uss-dissertation/assets/riyadh_boundary' #link to asset: https://code.earthengine.google.com/?asset=projects/uss-dissertation/assets/riyadh_boundary
city_boundary = load_city_boundary(asset_id)
# Using CartoDB.Positron basemap for a clean, light background.
map_obj = geemap.Map(basemap='CartoDB.Positron')
boundary_style = {"color": "black", "fillColor": "00000000", "width": 2}
map_obj.addLayer(city_boundary, boundary_style, "Riyadh City Boundary")
# Centers on Riyadh coordinates with zoom level 12 for city-wide view.
map_obj.setCenter(46.75, 24.78, 12)

# Calculate and add layers
years = [ 2024] # initially i tested multiple years for comparison, but for the final version only 2024 is kept.
lst_layers = calculate_lst(city_boundary, years)

# Export LST layers as GeoTIFFs to Google Drive for local download
for year in years:
        ee.batch.Export.image.toDrive(
            image=lst_layers[year],
            description=f'Riyadh_LST_{year}',
            folder='Riyadh_Layers',
            fileNamePrefix=f'Riyadh_LST_EPSG20438_{year}',
            region=city_boundary.geometry(),
            scale=30,  # Landsat resolution
            crs='EPSG:20438', # UTM Zone 38N for metric accuracy as per https://epsg.io/20438
            maxPixels=1e9
        ).start()
        print(f"Exporting LST_{year} to Google Drive")

map_obj = add_multi_year_layers(map_obj, city_boundary, lst_layers)
map_obj